![Immagine di esempio](https://camo.githubusercontent.com/7fbec98ddbc1dc4186852d1c29487efd7b1eb820c8b6ef34e113fcde40746be2/68747470733a2f2f6d65646961706970652e6465762f696d616765732f6d6f62696c652f706f73655f747261636b696e675f66756c6c5f626f64795f6c616e646d61726b732e706e67)


# Settings

In [41]:
#%pip install mediapipe opencv-python

In [42]:
import mediapipe as mp
import cv2
import time
import csv
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score #accuracy score
import pickle #library to save the model

# Setup MediaPipe Holistic instance
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Setup drawing utility
mp_drawing = mp.solutions.drawing_utils


In [43]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic # Holistic model that predicts 543 landmarks

with open('military.pkl', 'rb') as f:
    model = pickle.load(f) #load the model
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [44]:
#Curl counter variables
left_counter = 0
right_counter = 0
left_stage = None  # None = not curling, 1 = curling
right_stage = None  # None = not curling, 1 = curling
max_angle = 0
min_angle = 180
max_angle2 = 0
min_angle2 = 180
left_military_stage = "Down"
right_military_stage = "Down"
status_change = False
left_military_action = "Rest"
right_military_action = "Rest"

# Functions


In [45]:
def calculate_angle(a,b,c):
    a = np.array(a) #First
    b = np.array(b) #Mid
    c = np.array(c) #End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
    
    return angle

In [46]:
#def compute_score():
    # Military Press
    # The arm angle has to be between max 165-170 degrees and min 40-50 degrees (from test)
    # The elbow angle has to be between max 135-155 degrees and min 40-50 degrees (da 35 a 40 è un po meno ok ma comunque ok)
    # 167-180 is ok for the wrist angle
    

In [47]:
#hand speed

In [48]:
def status_check(left_arm_angle, status):
    
    
    if left_arm_angle > 160:
        stage = "Up"
    if left_arm_angle < 50 and status == "Up":
        stage = "Down"
        status_change = True
        
        
    
    return stage, status_change

# - Curl counter

In [49]:
# cap = cv2.VideoCapture(0)

# with holistic as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # MediaPipe Holistic detection
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False
#         results = pose.process(image)
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         # Extract elbow landmarks
#         try:
#             landmarks = results.pose_landmarks.landmark
#             left_elbow = [landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x,
#                           landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]
#             right_elbow = [landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x,
#                            landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
#             left_wrist = [landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].x,
#                           landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].y]
#             right_wrist = [landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].x,
#                            landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]
#             left_shoulder = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,
#                              landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
#             right_shoulder = [landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,
#                               landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]

#             # Calculate angles
#             left_elbow_angle = calculate_angle(
#                 left_shoulder, left_elbow, left_wrist)
#             right_elbow_angle = calculate_angle(
#                 right_shoulder, right_elbow, right_wrist)

#             # Visualize angles
#             cv2.putText(image, str(left_elbow_angle), tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

#             cv2.putText(image, str(right_elbow_angle), tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

#             # Curl counter logic for left elbow
#             if left_elbow_angle > 160:
#                 left_stage = "down"

#             if left_elbow_angle < 30 and left_stage == 'down':
#                 left_stage = "up"
#                 left_counter += 1
#                 print("Left Elbow Counter:", left_counter)

#             # Curl counter logic for right elbow
#             if right_elbow_angle > 160:
#                 right_stage = "down"

#             if right_elbow_angle < 30 and right_stage == 'down':
#                 right_stage = "up"
#                 right_counter += 1
#                 print("Right Elbow Counter:", right_counter)

#         except:
#             pass

#         # Render counters
#         cv2.rectangle(image, (10, 400), (210, 480), (245, 117, 16), -1)

#         cv2.putText(image, 'LEFT ARM', (20, 430),
#                     cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
#         cv2.putText(image, str(left_counter), (50, 455),
#                     cv2.FONT_HERSHEY_PLAIN, 1.7, (255, 255, 255), 2, cv2.LINE_AA)
#         cv2.putText(image, 'Stage: ' + str(left_stage), (20, 470),
#                     cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

#         cv2.putText(image, 'RIGHT ARM', (120, 430),
#                     cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
#         cv2.putText(image, str(right_counter), (150, 455),
#                     cv2.FONT_HERSHEY_PLAIN, 1.7, (255, 255, 255), 2, cv2.LINE_AA)
#         cv2.putText(image, 'Stage: ' + str(right_stage), (120, 470),
#                     cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

#         # Render landmarks
#         # 1. Draw face landmarks
#         mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
#                                   mp_drawing.DrawingSpec(
#                                       color=(80, 110, 10), thickness=1, circle_radius=1),
#                                   mp_drawing.DrawingSpec(
#                                       color=(80, 256, 121), thickness=1, circle_radius=1)
#                                   )

#         # 2. Right hand
#         mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(
#                                       color=(80, 22, 10), thickness=2, circle_radius=4),
#                                   mp_drawing.DrawingSpec(
#                                       color=(80, 44, 121), thickness=2, circle_radius=2)
#                                   )

#         # 3. Left Hand
#         mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(
#                                       color=(121, 22, 76), thickness=2, circle_radius=4),
#                                   mp_drawing.DrawingSpec(
#                                       color=(121, 44, 250), thickness=2, circle_radius=2)
#                                   )

#         # 4. Pose Detections
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(
#                                       color=(245, 117, 66), thickness=2, circle_radius=4),
#                                   mp_drawing.DrawingSpec(
#                                       color=(245, 66, 230), thickness=2, circle_radius=2)
#                                   )

#         cv2.imshow('MediaPipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

# cap.release()
# cv2.destroyAllWindows()

# Military Press


In [50]:
cap = cv2.VideoCapture(0)

start_time = None
right_start_time = None
left_down_movement_time = None
right_down_movement_time = None
left_up_movement_time = None
right_up_movement_time = None

with holistic as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract elbow landmarks
        # try:
        landmarks = results.pose_landmarks.landmark
        #13, 14
        left_elbow = [landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW.value].y]
        right_elbow = [landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].x,
                        landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW.value].y]
        #15, 16
        left_wrist = [landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].y]
        right_wrist = [landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].x,
                        landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]
        #11, 12
        left_shoulder = [landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,
                            landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
        right_shoulder = [landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,
                            landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
        #23, 24
        left_hip = [landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].x,
                    landmarks[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
        right_hip = [landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_holistic.PoseLandmark.RIGHT_HIP.value].y]
        #17, 18
        left_pinky = [landmarks[mp_holistic.PoseLandmark.LEFT_PINKY.value].x,
                        landmarks[mp_holistic.PoseLandmark.LEFT_PINKY.value].y]
        right_pinky = [landmarks[mp_holistic.PoseLandmark.RIGHT_PINKY.value].x,
                        landmarks[mp_holistic.PoseLandmark.RIGHT_PINKY.value].y]
        #19, 20
        left_index = [landmarks[mp_holistic.PoseLandmark.LEFT_INDEX.value].x,
                        landmarks[mp_holistic.PoseLandmark.LEFT_INDEX.value].y]
        right_index = [landmarks[mp_holistic.PoseLandmark.RIGHT_INDEX.value].x,
                        landmarks[mp_holistic.PoseLandmark.RIGHT_INDEX.value].y]
        

        # Calculate angles
        # Elbows angles
        left_elbow_angle = calculate_angle(
            left_shoulder, left_elbow, left_wrist)
        right_elbow_angle = calculate_angle(
            right_shoulder, right_elbow, right_wrist)
        # Arms angles relative to the body
        left_arm_angle = calculate_angle(
            left_elbow, left_shoulder, left_hip)
        right_arm_angle = calculate_angle(
            right_elbow, right_shoulder, right_hip)
        # Wrists angles
        right_index_wrist_angle = calculate_angle(
            right_index, right_wrist, right_elbow)
        right_pinky_wrist_angle = calculate_angle(
            right_pinky, right_wrist, right_elbow)
        left_index_wrist_angle = calculate_angle(
            left_index, left_wrist, left_elbow)
        left_pinky_wrist_angle = calculate_angle(
            left_pinky, left_wrist, left_elbow)
        
        # stage, status_change = status_check(left_arm_angle, stage)
        # print(stage)
        
        if left_arm_angle > 160 and left_military_stage == "Down":
            left_military_stage = "Up"
            status_change = True
            left_military_action = "Rest"
        if left_arm_angle <160 and left_arm_angle > 50:
            left_military_action = "Movement"
            status_change = False
        if left_arm_angle < 50 and left_military_stage == "Up":
            left_military_stage = "Down"
            status_change = True
            left_military_action = "Rest"
        # if status_change:
        #     print(stage)
        if left_military_stage == "Up" and left_military_action == "Movement":
            if start_time is None:
                start_time = time.time()
            #print(stage, left_military_action)
        if left_military_stage == "Down" and left_military_action == "Rest":
            if start_time is not None:
                time_elapsed = time.time() - start_time
                start_time = None
                #status_change = False
                left_down_movement_time = time_elapsed
                print(left_military_stage, left_military_action, time_elapsed)
        if left_military_stage == "Down" and left_military_action == "Movement":
            if start_time is None:
                start_time = time.time()
            #print(stage, action)
        if left_military_stage == "Up" and left_military_action == "Rest":
            if start_time is not None:
                time_elapsed = time.time() - start_time
                start_time = None
                #status_change = False #
                left_up_movement_time = time_elapsed
                print(left_military_stage, left_military_action, time_elapsed)
                
                
        if right_arm_angle > 160 and right_military_stage == "Down":
            right_military_stage = "Up"
            status_change = True
            right_military_action = "Rest"
        if right_arm_angle <160 and right_arm_angle > 50:
            right_military_action = "Movement"
            status_change = False
        if right_arm_angle < 50 and right_military_stage == "Up":
            right_military_stage = "Down"
            status_change = True
            right_military_action = "Rest"
        if right_military_stage == "Up" and right_military_action == "Movement":
            if right_start_time is None:
                right_start_time = time.time()
            #print(stage, action)
        if right_military_stage == "Down" and right_military_action == "Rest":
            if right_start_time is not None:
                right_time_elapsed = time.time() - right_start_time
                right_start_time = None
                #status_change = False
                right_down_movement_time = right_time_elapsed
                print(right_military_stage, right_military_action, right_time_elapsed)
        if right_military_stage == "Down" and right_military_action == "Movement":
            if right_start_time is None:
                right_start_time = time.time()
            #print(stage, action)
        if right_military_stage == "Up" and right_military_action == "Rest":
            if right_start_time is not None:
                right_time_elapsed = time.time() - right_start_time
                right_start_time = None
                #status_change = False #
                right_up_movement_time = right_time_elapsed
                print(right_military_stage, right_military_action, right_time_elapsed)
        
        
        # Calculate the difference between the speed of the right and left arm
        if right_down_movement_time is not None and left_down_movement_time is not None:
            down_speed_difference = abs(right_down_movement_time - left_down_movement_time)
            #print("Speed difference: ", speed_difference)
            if down_speed_difference < 0.5:
                down_coordination = "Correct"
                print("Correct coordination for down movement")
            else:
                down_coordination = "Incorrect"
            
        if right_up_movement_time is not None and left_up_movement_time is not None:
            up_speed_difference = abs(right_up_movement_time - left_up_movement_time)
            #print("Speed difference: ", speed_difference)
            if up_speed_difference < 0.5:
                up_coordination = "Correct"
                print("Correct coordination for up movement")
            else:
                up_coordination = "Incorrect"
        
        # Calculate the difference between the time of the movement up and down (right arm)
        if right_down_movement_time is not None and right_up_movement_time is not None:
            right_movement_time = right_down_movement_time - right_up_movement_time
            #print("Movement time: ", movement_time)
            right_down_movement_time = None
            right_up_movement_time = None
            if right_movement_time > 0.5:
                timing = "Correct"
                print("Correct timing for right arm")
        
        # Calculate the difference between the time of the movement up and down (left arm)
        if left_down_movement_time is not None and left_up_movement_time is not None:
            left_movement_time = left_down_movement_time - left_up_movement_time
            #print("Movement time: ", movement_time)
            left_down_movement_time = None
            left_up_movement_time = None
            if left_movement_time > 0.5:
                timing = "Correct"
                print("Correct timing for left arm")
        
        right_down_movement_time = None
        right_up_movement_time = None
        left_down_movement_time = None
        left_up_movement_time = None
        
        # Print max and min angles for understanding the range of the correct angles
        # if left_elbow_angle > max_angle:
        #     max_angle = left_elbow_angle
        #     print("Max left elbow angle: ", max_angle)
        # if left_elbow_angle < min_angle:
        #     min_angle = left_elbow_angle
        #     print("Min left elbow angle: ", min_angle)
        # if left_arm_angle > max_angle:
        #     max_angle = left_arm_angle
        #     print("Max left arm angle: ", max_angle)
        # if left_arm_angle < min_angle:
        #     min_angle = left_arm_angle
        #     print("Min left arm angle: ", min_angle)
        
        # Extract Pose landmarks for AI sulution
        # pose = results.pose_landmarks.landmark       
        # pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())         
        # # Extract Face landmarks
        # face = results.face_landmarks.landmark
        # face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())        
        # # Concate rows
        # row = pose_row+face_row
        
        # # Make Detections for AI solution
        # x = pd.DataFrame([row])
        # model_class = model.predict(x)[0] #class of the model
        # model_prob = model.predict_proba(x)[0] #probability of each class
        # #print(model_class, model_prob)
        # actual_stage = model_class
        
    

        # Visualize angles
        cv2.putText(image, str(left_elbow_angle), tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.putText(image, str(right_elbow_angle), tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.putText(image, str(left_arm_angle), tuple(np.multiply(left_shoulder, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.putText(image, str(right_arm_angle), tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            #cv2.putText(image, str(right_wrist_angle), tuple(np.multiply(right_wrist, [640, 480]).astype(int)),
              #          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        # except:
        #     pass


        # Render landmarks
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                  mp_drawing.DrawingSpec(
                                      color=(80, 110, 10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(
                                      color=(80, 256, 121), thickness=1, circle_radius=1)
                                  )

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(
                                      color=(80, 22, 10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(
                                      color=(80, 44, 121), thickness=2, circle_radius=2)
                                  )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(
                                      color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(
                                      color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(
                                      color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(
                                      color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('MediaPipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Up Rest 0.43828296661376953
Up Rest 0.5322690010070801
Down Rest 0.4323856830596924
Down Rest 0.3070993423461914
Correct coordination for down movement
Up Rest 0.31277942657470703
Up Rest 0.31277942657470703
Correct coordination for up movement
Down Rest 0.3120114803314209
Down Rest 0.37380313873291016
Up Rest 0.5262997150421143
Up Rest 0.4566924571990967
Correct coordination for up movement
Down Rest 0.44342923164367676
Down Rest 0.44342923164367676
Correct coordination for down movement
Up Rest 0.37201833724975586
Up Rest 0.37201833724975586
Correct coordination for up movement
Down Rest 0.31234169006347656
Down Rest 0.3729085922241211
Up Rest 0.372882604598999
Up Rest 0.372882604598999
Correct coordination for up movement
Down Rest 0.2816333770751953
Down Rest 0.3437831401824951
Up Rest 0.5628931522369385
Up Rest 0.5647237300872803
Down Rest 0.9052324295043945
Down Rest 1.719127893447876
Up Rest 0.2821025848388672
Up Rest 0.25330114364624023
Down Rest 0.2469029426574707
Down Rest 0.